### Scraper for Stage-Toyama website

In [16]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests

In [17]:
def get_chrome_Driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    return driver

def get_soup(URL):
    r = requests.get(URL)
    if r:
        driver = get_chrome_Driver()
        driver.get(URL)
        htmlContent = driver.page_source
        soup =  BeautifulSoup(htmlContent, 'html.parser')
        driver.close()
        return soup
    else:
        print("Error Fetching Soup, Scraping Ended")
        return False

def get_link_from_soup(soup):
    link_list = set()
    wanted_div = soup.findAll("ul", {"class":"p-postList -type-card -pc-col3 -sp-col1"})[0]
    wanted_article = wanted_div.findAll('a', href=True)
    for i in wanted_article:
        link_list.add(i["href"])
    return link_list

def save_link_as_txt(link_list):
    file_path = "links/stage-toyama.txt"
    # if file exist, get its content, merge to existing set and then overwrite
    if os.path.isfile(file_path):
        with open(file_path, 'r+', encoding="utf-8") as file:
            # get existing links in file
            old_links = set(file.readlines())
            # remove duplicates 
            link_list = link_list - old_links
            file.seek(0)
            for i in link_list:
                file.write(i+"\n")
            file.truncate()
    else: # if not, make new file
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, "w", encoding="utf-8") as file:
            for i in link_list:
                file.write(i+"\n")
    print(f"Links Saved in {file_path}")
    return True

In [18]:
all_links = set()
for i in range(1,20):
    soup = get_soup(f"https://stage-toyama.jp/category/garden/page/{i}/")
    if soup:
        all_links.update(get_link_from_soup(soup))
    else:
        break

save_link_as_txt(all_links)

Error Fetching Soup


True